In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cpu")

In [3]:
## Full rank matrix
A = torch.randn(8, 8)
A

tensor([[ 0.3962,  0.1699,  0.8402, -0.5260, -1.0450, -0.8706,  0.6028,  1.9461],
        [-3.2164,  0.6334,  0.1591,  1.3635,  1.2410,  0.8161,  0.0162,  0.9157],
        [-0.2821, -1.6466,  1.9572,  0.1952, -1.0894,  1.2173, -1.3713,  1.5161],
        [-2.4991, -2.2155, -0.3538,  0.6231,  0.1524,  1.0669, -0.8046, -2.3557],
        [-1.0083, -1.1730, -1.0936,  1.1958, -1.2631, -0.5616, -0.9239, -1.4451],
        [ 0.2663, -1.7632, -1.7054, -0.5899, -0.3153,  1.6183, -0.4723, -1.7493],
        [-1.2373, -0.3984,  2.8546, -2.2133,  1.4361, -1.5013,  0.6216,  0.7773],
        [-0.4651,  0.5786, -1.4568,  0.8709, -0.0620,  1.6008,  0.2742, -0.5049]])

In [4]:
### pair up (1,2), (3,4), ...
diag0 = torch.randn(4, 2, 2)
diag0

tensor([[[ 0.0346, -1.3588],
         [-0.9190,  0.3280]],

        [[ 0.9093,  1.8845],
         [ 0.8357,  0.8607]],

        [[ 0.6990,  0.2257],
         [-0.4755, -0.1515]],

        [[-0.2784,  0.0574],
         [ 0.2791, -0.7175]]])

In [5]:
torch.block_diag(*diag0)

tensor([[ 0.0346, -1.3588,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.9190,  0.3280,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.9093,  1.8845,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.8357,  0.8607,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.6990,  0.2257,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000, -0.4755, -0.1515,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.2784,  0.0574],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.2791, -0.7175]])

In [6]:
### Pair up (1, 3), (2,4)
## that means rearrange as.. 1,3,2,4 and apply block diagonal

def get_pair(dim, step=1):
    assert 2**int(np.log2(dim)) == dim , "The dim must be power of 2"
    assert isinstance(step, int), "Step must be integer"
    
    blocks = (2**step)
    range_ = dim//blocks
#     print(range_, blocks)
    adder_ = torch.arange(0, range_)*blocks
#     print(adder_)
    
    pairs_ = torch.Tensor([0, blocks//2])
    repeat_ = torch.arange(0, blocks//2).reshape(-1,1)
#     print(pairs_)
#     print(repeat_)
    
    block_map = (pairs_+repeat_).reshape(-1)
#     print(block_map)
    
#     print(block_map+adder_.reshape(-1,1))
    reorder_for_pair = (block_map+adder_.reshape(-1,1)).reshape(-1)
    indx = reorder_for_pair.type(torch.long)
    rev_indx = torch.argsort(indx)
    
    return indx, rev_indx

In [7]:
get_pair(8, 1)

(tensor([0, 1, 2, 3, 4, 5, 6, 7]), tensor([0, 1, 2, 3, 4, 5, 6, 7]))

In [8]:
get_pair(8, 2)

(tensor([0, 2, 1, 3, 4, 6, 5, 7]), tensor([0, 2, 1, 3, 4, 6, 5, 7]))

In [9]:
get_pair(8, 3)

(tensor([0, 4, 1, 5, 2, 6, 3, 7]), tensor([0, 2, 4, 6, 1, 3, 5, 7]))

In [10]:
mat = torch.arange(8, dtype=torch.float).reshape(-1, 1) + torch.arange(8, dtype=torch.float).reshape(1, -1)*0.1
mat

tensor([[0.0000, 0.1000, 0.2000, 0.3000, 0.4000, 0.5000, 0.6000, 0.7000],
        [1.0000, 1.1000, 1.2000, 1.3000, 1.4000, 1.5000, 1.6000, 1.7000],
        [2.0000, 2.1000, 2.2000, 2.3000, 2.4000, 2.5000, 2.6000, 2.7000],
        [3.0000, 3.1000, 3.2000, 3.3000, 3.4000, 3.5000, 3.6000, 3.7000],
        [4.0000, 4.1000, 4.2000, 4.3000, 4.4000, 4.5000, 4.6000, 4.7000],
        [5.0000, 5.1000, 5.2000, 5.3000, 5.4000, 5.5000, 5.6000, 5.7000],
        [6.0000, 6.1000, 6.2000, 6.3000, 6.4000, 6.5000, 6.6000, 6.7000],
        [7.0000, 7.1000, 7.2000, 7.3000, 7.4000, 7.5000, 7.6000, 7.7000]])

In [11]:
diag0 = torch.randn(4, 2, 2)
diag0.requires_grad=True
bd0 = torch.block_diag(*diag0)
bd0

tensor([[ 1.0522,  0.4284,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 1.7190, -0.1481,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.4868, -0.7199,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.3865, -1.8837,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.9123, -0.9595,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  1.4009, -0.5164,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.5273, -0.6390],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.9789, -2.1394]],
       grad_fn=<CopySlices>)

In [12]:
diag1 = torch.randn(4, 2, 2)
bd1 = torch.block_diag(*diag1)
# bd1 = mat
ind1, rind1 = get_pair(8, 2)
bd1 = bd1[ind1]
bd1 = bd1[:,ind1]
bd1

tensor([[ 0.6826,  0.0000,  0.4099,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.8366,  0.0000,  0.2396,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.8492,  0.0000,  0.6970,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000, -1.2991,  0.0000, -1.5302,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.8082,  0.0000,  0.7110,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.7219,  0.0000,  0.4748],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.9419,  0.0000,  0.2931,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0733,  0.0000,  0.4922]])

In [13]:
(bd0@bd1)

tensor([[ 0.7182,  0.3584,  0.4313,  0.1027,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 1.1733, -0.1239,  0.7046, -0.0355,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.4134,  0.9352,  0.3393,  1.1016,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.3282,  2.4471, -0.2694,  2.8824,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.7372, -0.6926,  0.6486, -0.4556],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  1.1321, -0.3728,  0.9961, -0.2452],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  1.4385, -0.6858,  0.4476, -0.3145],
        [ 0.0000,  0.0000,  0.0000,  0.0000, -0.9220, -2.2962, -0.2869, -1.0530]],
       grad_fn=<MmBackward>)

In [14]:
diag2 = torch.randn(4, 2, 2)
bd2 = torch.block_diag(*diag2)
# bd2 = mat
ind2, rind2 = get_pair(8, 3)
# bd2 = bd2[ind2]
# bd2 = bd2[:, ind2]
bd2

tensor([[-1.4186,  1.9533,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.9419, -1.4526,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -1.0854,  1.3065,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  1.2680,  1.3777,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.1636, -1.3610,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000, -1.6920, -0.4377,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.5950, -0.2387],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.6184,  0.6304]])

In [15]:
get_pair(8, 3)

(tensor([0, 4, 1, 5, 2, 6, 3, 7]), tensor([0, 2, 4, 6, 1, 3, 5, 7]))

In [16]:
torch.scatter

<function _VariableFunctionsClass.scatter>

In [17]:
bd2

tensor([[-1.4186,  1.9533,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.9419, -1.4526,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -1.0854,  1.3065,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  1.2680,  1.3777,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.1636, -1.3610,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000, -1.6920, -0.4377,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.5950, -0.2387],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.6184,  0.6304]])

In [18]:
bd2[:,rind2] = bd2.clone()

In [19]:
ind2, rind2

(tensor([0, 4, 1, 5, 2, 6, 3, 7]), tensor([0, 2, 4, 6, 1, 3, 5, 7]))

In [20]:
(bd0@bd1)@bd2

tensor([[-0.6813,  0.0000,  0.8823,  0.0000, -0.3380,  0.0000,  0.7049,  0.0000],
        [-1.7812,  0.0000,  2.4718,  0.0000, -0.8098,  0.0000,  0.8717,  0.0000],
        [ 1.4674,  0.0000, -2.1660,  0.0000,  1.0286,  0.0000,  1.9609,  0.0000],
        [ 1.8395,  0.0000, -2.9136,  0.0000,  3.9473,  0.0000,  3.6191,  0.0000],
        [ 0.0000,  1.2925,  0.0000, -0.7002,  0.0000,  0.1042,  0.0000, -0.4420],
        [ 0.0000,  0.8160,  0.0000, -1.3776,  0.0000,  0.4410,  0.0000, -0.3923],
        [ 0.0000,  1.3957,  0.0000, -1.6576,  0.0000,  0.0718,  0.0000, -0.3051],
        [ 0.0000,  3.7342,  0.0000,  2.2599,  0.0000, -0.8219,  0.0000, -0.5954]],
       grad_fn=<MmBackward>)

In [21]:
# indices

In [22]:
# torch.argsort(indices) ## this is reverse index

In [23]:
def get_pair_indices(dim, step=1):
    assert 2**int(np.log2(dim)) == dim , "The dim must be power of 2"
    assert isinstance(step, int), "Step must be integer"
    
    blocks = (2**step)
    range_ = dim//blocks
#     print(range_, blocks)
    adder_ = torch.arange(0, range_)*blocks
#     print(adder_)
    
    pairs_ = torch.Tensor([0, blocks//2])
    repeat_ = torch.arange(0, blocks//2).reshape(-1,1)
#     print(pairs_)
#     print(repeat_)
    
    block_map = (pairs_+repeat_).reshape(-1)
#     print(pairs_+repeat_)
    
#     print(block_map+adder_.reshape(-1,1))
    reorder_for_pair = (block_map+adder_.reshape(-1,1)).reshape(-1)
    indx = reorder_for_pair.type(torch.long)
#     rev_indx = torch.argsort(indx)
    
#     print(indx.reshape(-1, 2))
    
#     return indx, rev_indx
    indx = indx.reshape(-1, 2)
    
    map_idx = []
    for idx in indx:
        map_idx.append((idx[0], idx[0]))
        map_idx.append((idx[0], idx[1]))        
        map_idx.append((idx[1], idx[0]))        
        map_idx.append((idx[1], idx[1]))        
    map_idx = torch.LongTensor(map_idx)
#     print(map_idx)
    
    return map_idx

In [24]:
map_idx = get_pair_indices(8, 2)

In [25]:
map_idx.shape

torch.Size([16, 2])

In [26]:
w = torch.randn(map_idx.shape[0],1)

In [27]:
z = torch.zeros(8,8)
z[map_idx.split(1, dim=1)] = w
z

tensor([[ 0.0192,  0.0000,  0.2482,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  1.2985,  0.0000, -0.9498,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.2421,  0.0000, -0.8775,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000, -1.6119,  0.0000, -0.2002,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000, -0.3437,  0.0000, -0.9541,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.2826,  0.0000, -0.7138],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.4736,  0.0000, -0.8852,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.6394,  0.0000, -1.2892]])

In [28]:
torch.nonzero(mat, as_tuple=True)

(tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3,
         3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6,
         6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7]),
 tensor([1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0,
         1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0,
         1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7]))

### create 3 matrices with the factorization and multiply to approximate given matrix

In [40]:
A = torch.randn(8, 8)#*5
# A.abs_()

In [41]:
w0, w1, w2 = [torch.randn(16, 1, requires_grad=True) for _ in range(3)]

In [42]:
i0, i1, i2 = [get_pair_indices(8, i+1) for i in range(3)]

In [43]:
# optimizer = torch.optim.Adam([w0, w1, w2], lr=0.001)
optimizer = torch.optim.SGD([w0, w1, w2], lr=0.1)

mse = nn.MSELoss()
def mae(A, B):
    return torch.abs(A-B).mean()

In [44]:
### forward propagation
for i in range(10000):
    W0, W1, W2 = [torch.zeros(8, 8) for _ in range(3)]
    W0[i0.split(1, dim=1)] = w0
    W1[i1.split(1, dim=1)] = w1
    W2[i2.split(1, dim=1)] = w2

    W = (W0@W1)@W2
    loss = mse(W,A)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i%1000 == 0:
        print(f"The MSE loss is : {float(mse(W,A))}")
#         print(f"The MAE loss is : {float(mae(W,A))}")

The MSE loss is : 3.6087985038757324
The MSE loss is : 0.7655029892921448
The MSE loss is : 0.37236547470092773
The MSE loss is : 0.28449660539627075
The MSE loss is : 0.2388807088136673
The MSE loss is : 0.22659023106098175
The MSE loss is : 0.2215479016304016
The MSE loss is : 0.21875648200511932
The MSE loss is : 0.21700739860534668
The MSE loss is : 0.21356382966041565


In [45]:
W

tensor([[ 0.3993,  0.3801, -0.4785,  0.0519,  0.2730, -0.3165, -0.2511,  1.2175],
        [ 0.6930, -0.1058, -0.8303, -0.0145,  0.4738,  0.0881, -0.4358, -0.3388],
        [-0.0238, -2.2841,  0.0204, -0.0554, -0.0162,  1.9022,  0.0107, -1.2986],
        [ 2.1675, -1.0745, -1.8616, -0.0261,  1.4819,  0.8948, -0.9771, -0.6109],
        [ 0.5955,  0.3952,  1.2800,  1.7974,  0.7299, -0.7924,  1.2110, -0.8435],
        [-0.4154,  0.4828, -0.8929,  2.1961, -0.5091, -0.9681, -0.8447, -1.0305],
        [-0.1468,  0.0133, -0.2525,  0.0177, -0.1799, -0.0266, -0.2389, -0.0083],
        [-0.6535,  0.3553, -1.1246,  0.4743, -0.8010, -0.7125, -1.0639, -0.2226]],
       grad_fn=<MmBackward>)

In [46]:
A

tensor([[ 0.7685,  0.8957,  0.1331,  0.3989, -0.1294,  0.3699, -1.2668,  1.2201],
        [ 0.7227,  0.3880, -0.5730, -0.0298,  0.3511,  0.9222, -1.0123, -0.2755],
        [-0.0900, -2.1690,  0.0362,  0.6519,  0.4161,  2.0610,  0.4893, -1.2986],
        [ 2.0893, -1.1855, -2.1334, -0.8296,  1.6000,  0.7177, -0.4538, -0.6407],
        [ 0.8874,  0.8393,  0.7547,  1.2350,  1.0525, -1.0452,  1.4292, -1.5965],
        [-0.2712, -0.2711, -1.1783,  2.5720,  0.1748, -0.9558, -1.0259, -0.5940],
        [-0.7075, -1.0391, -0.2614,  0.3595,  0.3455, -0.3990, -0.2806,  0.2319],
        [-0.3533,  0.9251, -1.4940,  0.8523, -1.0598, -0.4341, -0.6624,  0.6012]])

In [47]:
W-A

tensor([[-3.6914e-01, -5.1569e-01, -6.1155e-01, -3.4696e-01,  4.0241e-01,
         -6.8636e-01,  1.0156e+00, -2.6459e-03],
        [-2.9716e-02, -4.9381e-01, -2.5735e-01,  1.5331e-02,  1.2271e-01,
         -8.3413e-01,  5.7644e-01, -6.3348e-02],
        [ 6.6237e-02, -1.1516e-01, -1.5824e-02, -7.0728e-01, -4.3231e-01,
         -1.5886e-01, -4.7856e-01,  3.1352e-05],
        [ 7.8234e-02,  1.1101e-01,  2.7179e-01,  8.0357e-01, -1.1811e-01,
          1.7705e-01, -5.2338e-01,  2.9798e-02],
        [-2.9192e-01, -4.4412e-01,  5.2533e-01,  5.6247e-01, -3.2261e-01,
          2.5287e-01, -2.1820e-01,  7.5303e-01],
        [-1.4417e-01,  7.5391e-01,  2.8545e-01, -3.7598e-01, -6.8396e-01,
         -1.2276e-02,  1.8115e-01, -4.3649e-01],
        [ 5.6072e-01,  1.0524e+00,  8.8915e-03, -3.4179e-01, -5.2536e-01,
          3.7235e-01,  4.1680e-02, -2.4026e-01],
        [-3.0024e-01, -5.6974e-01,  3.6937e-01, -3.7801e-01,  2.5880e-01,
         -2.7843e-01, -4.0153e-01, -8.2377e-01]], grad_fn=<SubBac

In [48]:
torch.det(W.data)

tensor(-0.0100)

In [49]:
torch.det(A.data)

tensor(77.0546)

### using double matrix

In [51]:
# A = torch.randn(8, 8)
# A.abs_()

In [52]:
w0, w1, w2 = [torch.randn(16, 1, requires_grad=True) for _ in range(3)]

w3, w4, w5 = [torch.randn(16, 1, requires_grad=True) for _ in range(3)]

In [53]:
i0, i1, i2 = [get_pair_indices(8, i+1) for i in range(3)]

In [54]:
# optimizer = torch.optim.Adam([w0, w1, w2, w3, w4, w5], lr=0.001)
optimizer = torch.optim.SGD([w0, w1, w2, w3, w4, w5], lr=0.1)

mse = nn.MSELoss()
def mae(A, B):
    return torch.abs(A-B).mean()

In [55]:
### forward propagation
for i in range(10000):
    W0, W1, W2, W3, W4, W5 = [torch.zeros(8, 8) for _ in range(6)]
    W0[i0.split(1, dim=1)] = w0
    W1[i1.split(1, dim=1)] = w1
    W2[i2.split(1, dim=1)] = w2
    
    W3[i0.split(1, dim=1)] = w3
    W4[i1.split(1, dim=1)] = w4
    W5[i2.split(1, dim=1)] = w5

    W = ((((W0@W1)@W2)@W3)@W4)@W5
    loss = mse(W,A)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i%1000 == 0:
        print(f"The MSE loss is : {float(mse(W,A))}")
#         print(f"The MAE loss is : {float(mae(W,A))}")

The MSE loss is : 5.389554977416992
The MSE loss is : 0.14279009401798248
The MSE loss is : 0.07612577080726624
The MSE loss is : 0.07401648163795471
The MSE loss is : 0.07178101688623428
The MSE loss is : 0.0686001181602478
The MSE loss is : 0.06434062123298645
The MSE loss is : 0.05949222669005394
The MSE loss is : 0.048408392816782
The MSE loss is : 0.03712528944015503


In [56]:
W

tensor([[ 0.7894,  1.0445,  0.1873,  0.2998, -0.0917,  0.3960, -1.3301,  0.9799],
        [ 0.7238, -0.0649, -0.5996,  0.1188,  0.1697,  0.7065, -1.0316,  0.1990],
        [ 0.2036, -2.1799,  0.2128,  0.5518,  0.1847,  1.9795,  0.3659, -1.4294],
        [ 2.0312, -0.9351, -2.1456, -0.7819,  1.6493,  0.9967, -0.4567, -0.6296],
        [ 0.9578,  0.5616,  0.9649,  1.1997,  1.2090, -1.2870,  1.0522, -1.4911],
        [-0.3245,  0.2382, -1.1977,  2.6212,  0.2019, -0.5683, -0.9120, -0.6433],
        [-0.6558, -0.9914, -0.2588,  0.3598,  0.4217, -0.2739, -0.3381,  0.2691],
        [-0.2944,  1.0302, -1.2514,  0.8598, -0.7786, -0.2188, -1.0460,  0.6972]],
       grad_fn=<MmBackward>)

In [57]:
A

tensor([[ 0.7685,  0.8957,  0.1331,  0.3989, -0.1294,  0.3699, -1.2668,  1.2201],
        [ 0.7227,  0.3880, -0.5730, -0.0298,  0.3511,  0.9222, -1.0123, -0.2755],
        [-0.0900, -2.1690,  0.0362,  0.6519,  0.4161,  2.0610,  0.4893, -1.2986],
        [ 2.0893, -1.1855, -2.1334, -0.8296,  1.6000,  0.7177, -0.4538, -0.6407],
        [ 0.8874,  0.8393,  0.7547,  1.2350,  1.0525, -1.0452,  1.4292, -1.5965],
        [-0.2712, -0.2711, -1.1783,  2.5720,  0.1748, -0.9558, -1.0259, -0.5940],
        [-0.7075, -1.0391, -0.2614,  0.3595,  0.3455, -0.3990, -0.2806,  0.2319],
        [-0.3533,  0.9251, -1.4940,  0.8523, -1.0598, -0.4341, -0.6624,  0.6012]])

In [58]:
W-A

tensor([[ 2.0916e-02,  1.4872e-01,  5.4238e-02, -9.9064e-02,  3.7652e-02,
          2.6179e-02, -6.3313e-02, -2.4019e-01],
        [ 1.1517e-03, -4.5297e-01, -2.6666e-02,  1.4861e-01, -1.8134e-01,
         -2.1570e-01, -1.9324e-02,  4.7450e-01],
        [ 2.9356e-01, -1.0891e-02,  1.7659e-01, -1.0010e-01, -2.3141e-01,
         -8.1566e-02, -1.2337e-01, -1.3081e-01],
        [-5.8044e-02,  2.5040e-01, -1.2226e-02,  4.7720e-02,  4.9333e-02,
          2.7901e-01, -2.9171e-03,  1.1062e-02],
        [ 7.0382e-02, -2.7765e-01,  2.1017e-01, -3.5275e-02,  1.5646e-01,
         -2.4174e-01, -3.7697e-01,  1.0543e-01],
        [-5.3297e-02,  5.0929e-01, -1.9387e-02,  4.9201e-02,  2.7030e-02,
          3.8755e-01,  1.1389e-01, -4.9242e-02],
        [ 5.1718e-02,  4.7668e-02,  2.6016e-03,  2.7749e-04,  7.6236e-02,
          1.2509e-01, -5.7526e-02,  3.7130e-02],
        [ 5.8843e-02,  1.0510e-01,  2.4254e-01,  7.5387e-03,  2.8116e-01,
          2.1526e-01, -3.8362e-01,  9.6014e-02]], grad_fn=<SubBac

In [59]:
torch.det(W.data)

tensor(-5.5777)

In [60]:
torch.det(A.data)

tensor(77.0546)